In [1]:
from abc import ABCMeta, abstractmethod
from collections import namedtuple
import numpy as np

import plotly.graph_objects as go 
import plotly.express as px 
import plotly.io as pio
from plotly.subplots import make_subplots

from scipy.optimize import minimize
from easymh import mh

In [2]:
class Law(metaclass=ABCMeta):
    @staticmethod
    @abstractmethod
    def sample(n, d):
        pass

    @staticmethod
    @abstractmethod
    def loglikely(n, d, k):
        pass
   
    @staticmethod
    def likelihood(n, d, k):
        return np.exp(loglikely(n, d, k))


class Bin(Law):
    def sample(n, d):
        return np.random.binomial(n, d)
    
    def loglikely(n, d, k):
        return k*np.log(d) + (n-k)*np.log(1-d)
 
       
class Poi(Law):
    def sample(n, d):
        return np.random.poisson(n*d)
    
    def loglikely(n, d, k):
        return k*np.log(n*d) - n*d - np.sum(np.log(np.arange(k)+1))
    

class Gau(Law):
    def sample(n, d=1):
        return n * (1 + 0.1*np.random.randn())
    
    def loglikely(n, d, k):
        return -50 * np.log(k/n)**2 

In [358]:
def variation1(x):
    return np.mean(np.abs(np.diff(x)))


def variation2(x):
    return np.sqrt(np.mean(np.diff(x)**2))


def elastic_net(x, mu=1):
    return (variation1(x) + mu*variation2(x)) / (1+mu)

In [137]:
Region = namedtuple('Region', 'S I R Q')
Epidemic = namedtuple('Epidemic', 'S I R Q')


class Sample:
    def __init__(self, epidemic, ts, ms, ns, law, seed=None):
        if seed is not None:
            np.random.seed(seed)
        
        positive = np.zeros_like(ts)
        for i, (t, m, n) in enumerate(zip(ts, ms, ns)):
            positive[i] = law.sample(n, epidemic.I[t]/m)
            
        self.t = ts
        self.m = ms
        self.n = ns
        self.positive = positive
        self._law = law
        
    def __repr__(self):
        return " t: {} \n m: {} \n n: {} \n positive: {}".format(self.t, self.m, self.n, self.positive)
    
    def plot(self, fig):
        fig.add_scatter(
            x=self.t, y=self.positive / self.n * self.m, 
            mode="markers", marker_symbol=1, name="Guessed", hovertemplate="%{y}"
        )
        return fig
    

class Confirmed:
    def __init__(self, epidemic, ts, law=Poi, seed=None):
        if seed is not None:
            np.random.seed(seed)
        
        c = np.zeros_like(ts)
        for i, t in enumerate(ts):
            c[i] = law.sample(epidemic.Q[t], 1)

        self.t = ts
        self.c = c
        
    def __repr__(self):
        return " t: {} \n c: {}".format(self.t, self.c)
    
    def plot(self, fig):
        fig.add_scatter(
            x=self.t, y=self.c, 
            mode="markers", marker_symbol=2, name="Confirmed", hovertemplate="%{y}"
        )
        return fig

In [179]:
class JumpProcess:
    def __init__(self, start, amplitude, wait, horizon, seed=None):
        if seed is not None:
            np.random.seed(seed)
            
        process = np.zeros(horizon)
        t = 0
        while t < horizon:
            T = int(np.random.exponential(wait))
            process[t:t+T] = start
            start *= (1 + np.random.choice([-1, 1]) * amplitude)
            t += T
            
        self.value = process
            
    def plot(self):
        fig = px.scatter(x=range(len(self.value)), y=self.value, hovertemplate="%{y}", line_shape='hv')
        fig.update_yaxes(type="log")
        return fig

In [283]:
class SIRt:
    def __init__(self, beta, gamma, dt=1):
        if np.isscalar(gamma):
            gamma = gamma * np.ones_like(beta)
        elif len(beta) != len(gamma):
            raise Exception("Dimensions not equal.")
            
        self.beta = beta * dt
        self.gamma = gamma * dt
        
    def __repr__(self):
        fig = go.Figure()
        fig.add_scatter(x=np.arange(len(self.beta)), y=self.beta, line_shape='hv', name='β', hovertemplate="%{y}")
        fig.add_scatter(x=np.arange(len(self.gamma)), y=self.gamma, line_shape='hv', name='γ', hovertemplate="%{y}")
        fig.update_yaxes(type="log")
        fig.show()
        return ""
    
    def r0(self):
        return self.beta / self.gamma
    
    def estimate(self, region):
        T = len(self.beta)
        S = np.zeros(T+1)
        I = np.zeros(T+1)
        R = np.zeros(T+1)
        S[0] = region.S
        I[0] = region.I
        R[0] = region.R
        
        for t in range(T):
            M = S[t] + I[t] + R[t]
            a, b = self.beta[t]*S[t]*I[t]/M, self.gamma[t]*I[t]
            S[t+1] = S[t] - a
            I[t+1] = I[t] + a - b
            R[t+1] = R[t] + b        
        
        return Epidemic(S, I, R, None)

    def predict(self, region, T):
        S = np.zeros(T+1)
        I = np.zeros(T+1)
        R = np.zeros(T+1)
        S[0] = region.S
        I[0] = region.I
        R[0] = region.R
        
        for t in range(T):
            M = S[t] + I[t] + R[t]
            a, b = self.beta[-1]*S[t]*I[t]/M, self.gamma[-1]*I[t]   # replaced `t` with `-1`
            S[t+1] = S[t] - a
            I[t+1] = I[t] + a - b
            R[t+1] = R[t] + b        
        
        return Epidemic(S, I, R, None)
    
    @staticmethod
    def plot(epidemic, T0=0, T=None, line=None, fig=None):
        if fig is None:
            fig = go.Figure()
            fig.update_layout(margin=dict(b=0, l=0, r=0, t=25))
        if T is None:    
            T = len(epidemic.S) - 1 + T0
        fig.add_scatter(x=np.arange(T0, T+1), y=epidemic.S.astype(int), name="Susceptible", hovertemplate="%{y}")
        fig.add_scatter(x=np.arange(T0, T+1), y=epidemic.I.astype(int), name="Infectious", hovertemplate="%{y}")
        fig.add_scatter(x=np.arange(T0, T+1), y=epidemic.R.astype(int), name="Removed", hovertemplate="%{y}")
        return fig
    
    def project(self, region, T):
        epidemic = self.estimate(region)
        fig = self.plot(epidemic, T0=0)
        region = Region(epidemic.S[-1], epidemic.I[-1], epidemic.R[-1], None)
        epidemic = self.predict(region, T)
        self.plot(epidemic, T0=len(self.beta), fig=fig)
        return fig

In [390]:
def loglikely(epidemic, sample, law):
    ms = sample.m
    ns = sample.n
    ds = epidemic.I[sample.t] / ms
    ks = sample.positive
    return sum(law.loglikely(n, d, k) for n, d, k in zip(ns, ds, ks)) / len(sample.t)


def likelihood(epidemic, sample, law):
    return np.exp(loglikely(epidemic, sample, law))


class InferSIRt:
    def __init__(self, law_s=Bin, penalty_b=variation1, weight_b=1, algo='map'):
        self.law_s = law_s
        self.penalty_b = penalty_b
        self.weight_b = weight_b
        self.algo = algo
        self.dynamic = None
        self.loglikely = None
        
    def __str__(self):
        fig = go.Figure()
        fig.add_scatter(x=np.arange(len(self.beta)), y=self.beta, line_shape='hv', name='β', hovertemplate="%{y}")
        fig.update_yaxes(type="log")
        fig.show()
        return "γ={}, loglikely={}".format(self.gamma, self.loglikely)

    def fit(self, region, sample, **kvarg):
        if self.algo == "map":
            self.fit_beta_gamma_map(region, sample, **kvarg)
        elif self.algo == "mcmc":
            self.fit_beta_gamma_mh(region, sample, **kvarg)
            
    def fit_beta_gamma_map(self, region, sample, **kvarg):
        def func(x):
            dynamic = SIRt(x[1:], x[0])
            epidemic = dynamic.estimate(region)
            ll = -loglikely(epidemic, sample, self.law_s)
            return ll + self.weight_b * self.penalty_b(x[1:])
        
        x0 = 0.2 * np.ones(sample.t[-1]+1)
        res = minimize(func, x0, method='nelder-mead', options={'xatol': 1e-8, 'disp': True, 'maxiter': 100000})
        self.dynamic = SIRt(res.x[1:], res.x[0])
        self.loglikely = -res.fun

    def fit_beta_gamma_mh(self, region, sample, method='naive', **kvarg):
        if 'width' not in kvarg:
            kvarg['width'] = 0.1
        
        def func(x):
            dynamic = SIRt(x[1:], x[0])
            epidemic = dynamic.estimate(region)
            l = likelihood(epidemic, sample, self.law_s)
            return l * np.exp(-self.weight_b * self.penalty_b(x[1:]))
        
        def func2(x):
            x = np.power(10, x)
            dynamic = SIRt(x[1:], x[0])
            epidemic = dynamic.estimate(region)
            l = likelihood(epidemic, sample, self.law_s) * np.prod(x)
            return l * np.exp(-self.weight_b * self.penalty_b(x[1:]))
            
        T = sample.t[-1]
        if method == 'naive':
            res, walker = mh([0.5]*(T+1), func, np.tile([0.01, 1], (T+1, 1)), **kvarg)
        elif method == 'mirror':
            res, walker = mh([0.5]*(T+1), func, np.tile([0.01, 1], (T+1, 1)), ascdes=(np.log, np.exp), **kvarg)
        elif method == 'repar':
            res, walker = mh([-1]*(T+1), func2, np.tile([-2, 0], (T+1, 1)), **kvarg)
            res = np.power(10, res)
            walker = np.power(10, walker)

        self.dynamic = SIRt(res[1:], res[0])
        self.loglikely = np.log(func(res))
        self.walker = walker       

In [335]:
T = 100
beta = JumpProcess(2, 0.2, 10, T, 0)
gamma = JumpProcess(1, 0.02, 10, T, 1)
dynamic = SIRt(beta.value, gamma.value, 0.1)
dynamic

In [336]:
city = Region(990, 10, 0, 0)
epidemic = dynamic.estimate(city)
fig = SIRt.plot(epidemic)
sample = Sample(epidemic, np.arange(T//10, T, T//10), 1000*np.ones(9), 10*np.ones(9), Bin, seed=0)
sample.plot(fig)

In [337]:
from hsir.sir import InferSIR, SIR
a = InferSIR()
a.fit(city, sample)

epi = SIR(a.beta, a.gamma).estimate(city, T)
fig = SIR.plot(epi)
#SIRt.plot(epidemic, fig=fig)
sample.plot(fig)

Optimization terminated successfully.
         Current function value: 7.000297
         Iterations: 66
         Function evaluations: 126


In [385]:
b = InferSIRt(penalty_b=variation1, weight_b=4)
b.fit(city, sample)
print(b.loglikely)
b.dynamic

Optimization terminated successfully.
         Current function value: 2.442134
         Iterations: 84825
         Function evaluations: 92557
-2.4421335287692005


In [350]:
dynamic

In [361]:
fig = SIRt.plot(b.dynamic.estimate(city))
sample.plot(fig)

In [346]:
fig = SIRt.plot(epidemic)
sample.plot(fig)

In [399]:
c = InferSIRt(penalty_b=variation1, weight_b=4, algo='mcmc')
c.fit(city, sample, method='mirror', move='u', width=0.1, N=100000, B=10000)
print(c.loglikely)
c.dynamic

-3.1986397737053562


In [400]:
fig = SIRt.plot(c.dynamic.estimate(city))
sample.plot(fig)

In [404]:
px.line(y=c.walker[1, ])